<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.3: Common Pass Idioms

**Prev: [FIRRTL AST Traversal](4.2_firrtl_ast_traversal.ipynb)**<br>
**Next: [A FIRRTL Transform Example](4.4_firrtl_add_ops_per_module.ipynb)**

### Adding statements
중첩된 DoPrim 표현식을 분할하여 다음과 같이 변환하는 패스를 작성한다고 가정합니다.
```
circuit Top:
  module Top :
  input x: UInt<3>
  input y: UInt<3>
  input z: UInt<3>
  output o: UInt<3>
  o <= add(x, add(y, z))
```
into this:
```
circuit Top:
  module Top :
  input x: UInt<3>
  input y: UInt<3>
  input z: UInt<3>
  output o: UInt<3>
  node GEN_1 = add(y, z)
  o <= add(x, GEN_1)
```

먼저 모든 문과 식에 대해 AST를 탐색해야 합니다. 그런 다음 DoPrim이 표시되면 모듈 본문에 새 DefNode를 추가하고 DoPrim 대신 해당 DefNode에 대한 참조를 삽입해야 합니다. 아래 코드는 이를 구현하고 정보 토큰을 유지합니다. '네임스페이스'는 [Namespace.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl/Namespace.scala)에 있는 유틸리티 함수입니다.

```scala
object Splitter extends Pass {
  def name = "Splitter!"
  /** Run splitM on every module **/
  def run(c: Circuit): Circuit = c.copy(modules = c.modules map(splitM(_)))

  /** Run splitS on the body of every module **/
  def splitM(m: DefModule): DefModule = m map splitS(Namespace(m))

  /** Run splitE on all children Expressions.
    * If stmts contain extra statements, return a Block containing them and 
    *    the new statement; otherwise, return the new statement. */
  def splitS(namespace: Namespace)(s: Statement): Statement = {
    val block = mutable.ArrayBuffer[Statement]()
    s match {
      case s: HasInfo => 
        val newStmt = s map splitE(block, namespace, s.info)
        block.length match {
          case 0 => newStmt
          case _ => Block(block.toSeq :+ newStmt)
        }
      case s => s map splitS(namespace)
  }

  /** Run splitE on all children expressions.
    * If e is a DoPrim, add a new DefNode to block and return reference to
    * the DefNode; otherwise return e.*/
  def splitE(block: mutable.ArrayBuffer[Statement], namespace: Namespace, 
             info: Info)(e: Expression): Expression = e map splitE(block, namespace, info) match {
    case e: DoPrim =>
      val newName = namespace.newTemp
      block += DefNode(info, newName, e)
      Ref(newName, e.tpe)
    case _ => e
  }
}
```
### Deleting statements
값이 리터럴인 모든 DefNode를 인라인하는 패스를 작성하여 다음과 같이 변환한다고 가정합니다.
```
circuit Top:
  module Top :
  input x: UInt<3>
  output o: UInt<4>
  node y = UInt(1)
  o <= add(x, y)
```
into this:
```
circuit Top:
  module Top :
  input x: UInt<3>
  output y: UInt<4>
  o <= add(x, UInt(1))
```

먼저 모든 문과 식에 대해 AST를 탐색해야 합니다. 그런 다음, 리터럴을 가리키는 DefNode가 표시되면 이를 해시맵에 저장하고 EmptyStmt를 반환해야 합니다(따라서 해당 DefNode 삭제). 그런 다음 삭제된 DefNode에 대한 참조를 볼 때마다 해당 리터럴을 삽입해야 합니다.

```scala
object Inliner extends Pass {
  def name = "Inliner!"
  /** Run inlineM on every module **/
  def run(c: Circuit): Circuit = c.copy(modules = c.modules map(inlineM(_)))

  /** Run inlineS on the body of every module **/
  def inlineM(m: DefModule): DefModule = m map inlineS(mutable.HashMap[String, Expression]())

  /** Run inlineE on all children Expressions, and then run inlineS on children statements.
    * If statement is a DefNode containing a literal, update values and
    *   return EmptyStmt; otherwise return statement. */
  def inlineS(values: mutable.HashMap[String, Expression])(s: Statement): Statement =
    s map inlineE(values) map inlineS(values) match {
      case d: DefNode => d.value match {
        case l: Literal =>
          values(d.name) = l
          EmptyStmt
        case _ => d
      }
      case o => o 
    }

  /** If e is a reference whose name is contained in values, 
    *   return values(e.name); otherwise run inlineE on all 
    *   children expressions.*/
  def inlineE(values: mutable.HashMap[String, Expression])(e: Expression): Expression = e match {
    case e: Ref if values.contains(e.name) => values(e.name)
    case _ => e map inlineE(values)
  }
}
```

### Add a Primop
유용할까요? [the firrtl repo](https://github.com/freechipsproject/firrtl)에 문제를 제출하여 [@azidar](https://github.com/azidar)에게 알려주세요!

### Swap a statement
유용할까요? [the firrtl repo](https://github.com/freechipsproject/firrtl)에 문제를 제출하여 [@azidar](https://github.com/azidar)에게 알려주세요!
